In [1]:
import getpass
user = getpass.getuser()
from omni.isaac.kit import SimulationApp
# Set the path below to your desired nucleus server
simulation_app = SimulationApp({"livesync_usd": f'omniverse://localhost/Users/{user}/isaac/sorting/scratch.usd'})
print("Hi")

Passing the following args to the base kit application:  ['-f', '/home/bjnortier/.local/share/jupyter/runtime/kernel-e86a13a5-c4d3-4cc0-98dd-d7e0f0ac4f36.json']
[Warning] [omni.kit.app.plugin] No crash reporter present, dumps uploading isn't available.
[Info] [carb] Logging to file: /home/bjnortier/.local/share/ov/pkg/isaac_sim-2021.2.1/kit/logs/Kit/Isaac-Sim/2021.2/kit_20220216_160044.log
[0.047s] [ext: omni.kit.pipapi-0.0.0] startup
[0.058s] [ext: omni.kit.pip_archive-0.0.0] startup
[0.062s] [ext: omni.usd.config-1.0.0] startup
[0.064s] [ext: omni.usd.libs-1.0.0] startup
[0.219s] [ext: omni.isaac.pip_archive-0.1.0] startup
[0.367s] [ext: omni.stats-0.0.0] startup
[0.369s] [ext: omni.gpu_foundation-0.0.0] startup
[0.375s] [ext: omni.assets.plugins-0.0.0] startup
[0.376s] [ext: omni.kit.loop-isaac-0.1.0] startup
[0.377s] [ext: omni.kit.async_engine-0.0.0] startup
[0.378s] [ext: omni.appwindow-1.0.0] startup
[0.388s] [ext: omni.client-0.1.0] startup
[0.397s] [ext: omni.kit.test-0.0.0] s

[3.948s] [ext: omni.kit.window.preferenceSimulation App Starting
Saving a temp livesync stage at  omniverse://localhost/Users/bjnortier/isaac/sorting/scratch.usd  ...Done.
Simulation App Startup Complete
Hi


In [2]:
from omni.isaac.core import World
from pxr import Gf, Sdf, UsdGeom, UsdShade
from omni.isaac.core.utils.prims import create_prim, get_prim_at_path
from omni.isaac.core.utils.rotations import euler_angles_to_quat
from omni.isaac.core.utils.viewports import set_camera_view
from omni.isaac.core.utils.stage import clear_stage
from omni.isaac.core.utils.nucleus import find_nucleus_server
from omni.isaac.core.materials.omni_glass import OmniGlass
from omni.isaac.core.prims.xform_prim import XFormPrim
from omni.isaac.core.utils.extensions import get_extension_path_from_name
from omni.isaac.core.utils.semantics import add_update_semantics
from omni.isaac.core.utils.prims import delete_prim
import omni.isaac.dr as dr
        

import omni
import carb
import random
import numpy as np
from omni.physx.scripts import utils

In [3]:
world = World(stage_units_in_meters = 0.01)
world.step()

In [8]:
CAMERA_DISTANCE = 100

def create_camera():
    camera_rig = UsdGeom.Xformable(create_prim("/World/CameraRig", "Xform"))
    camera = create_prim("/World/CameraRig/Camera", "Camera", position=np.array([0.0, 0.0, CAMERA_DISTANCE]))
    vpi = omni.kit.viewport.get_viewport_interface()
    vpi.get_viewport_window().set_active_camera(str(camera.GetPath()))
    viewport = omni.kit.viewport.get_default_viewport_window()

def create_environment():
    plate = create_prim(
        "/World/C",
        "Cube",
        position=np.array([0.0, -0.5, 0.0]),
        scale=np.array([100.0, 100.0, 1.0]),
        orientation=euler_angles_to_quat(np.array([0.0, 0.0, 0.0]), degrees=True),
        attributes={"primvars:displayColor": [(71.0 / 255.0, 165.0 / 255.0, 1.0)]},
    )
    utils.setCollider(plate, approximationShape="convexHull")
    
    
def create_light():
    create_prim("/World/LightRig", "Xform")
    create_prim(
        "/World/LightRig/Light",
        "SphereLight",
        position=np.array([0, 0, 200]),
        attributes={"radius": 50, "intensity": 5e4, "color": (1.0, 1.0, 1.0)}
    )
    
def create_dr():
#     dr.commands.RandomizeOnceCommand().do()
#     dr.commands.ToggleManualModeCommand().do()
    dr.commands.CreateRotationComponentCommand(
        prim_paths=["/World/LightRig"],
        min_range=(0.0, 0.0, -180.0),
        max_range=(0.0, 45.0, 180.0),
        seed=2866
    ).do()
    dr.commands.CreateRotationComponentCommand(
        prim_paths=["/World/CameraRig"],
        min_range=(0.0, 0.0, -180.0),
        max_range=(0.0, 45.0, 180.0),
        seed=2867
    ).do()
    return

world.clear()
create_camera()
create_environment()
create_light()
create_dr()
world.step()
world.step()

In [49]:
world.clear()